In [1]:
import pandas as pd


In [2]:
glassdoor_file = 'resources/glassdoor.csv'
df = pd.read_csv(glassdoor_file)
drop_col = df.columns
drop_col

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'hourly', 'employer_provided', 'min_salary', 'max_salary', 'avg_salary',
       'company_txt', 'job_state', 'same_state', 'age', 'python_yn', 'R_yn',
       'spark', 'aws', 'excel'],
      dtype='object')

In [3]:
b = ['Job Title', 'Job Description', 'Rating', 'Location', 'Headquarters', 'Size', 'Founded', 'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors', 'hourly', 'employer_provided', 'min_salary', 'max_salary', 'avg_salary', 'company_txt', 'job_state', 'same_state', 'age', 'python_yn', 'R_yn', 'spark', 'aws', 'excel']
df.drop(axis=1, columns=b, inplace=True)
df.head()

,Salary Estimate,Company Name
0,$53K-$91K (Glassdoor est.),Tecolote Research\n3.8
1,$63K-$112K (Glassdoor est.),University of Maryland Medical System\n3.4
2,$80K-$90K (Glassdoor est.),KnowBe4\n4.8
3,$56K-$97K (Glassdoor est.),PNNL\n3.8
4,$86K-$143K (Glassdoor est.),Affinity Solutions\n2.9


In [4]:
df['Company Name'].unique

<bound method Series.unique of 0                          Tecolote Research\n3.8
1      University of Maryland Medical System\n3.4
2                                    KnowBe4\n4.8
3                                       PNNL\n3.8
4                         Affinity Solutions\n2.9
                          ...                    
737                                      GSK\n3.9
738                               Eventbrite\n4.4
739           Software Engineering Institute\n2.6
740                             Numeric, LLC\n3.2
741             Riverside Research Institute\n3.6
Name: Company Name, Length: 742, dtype: object>

In [5]:
df.drop_duplicates(subset=['Company Name'],inplace=True)
df.reset_index(inplace=True, drop=True)
df

,Salary Estimate,Company Name
0,$53K-$91K (Glassdoor est.),Tecolote Research\n3.8
1,$63K-$112K (Glassdoor est.),University of Maryland Medical System\n3.4
2,$80K-$90K (Glassdoor est.),KnowBe4\n4.8
3,$56K-$97K (Glassdoor est.),PNNL\n3.8
4,$86K-$143K (Glassdoor est.),Affinity Solutions\n2.9
...,...,...
338,$79K-$127K (Glassdoor est.),SoftBank Robotics\n3.8
339,Employer Provided Salary:$120K-$140K,SkySync\n5.0
340,$65K-$113K (Glassdoor est.),"DatamanUSA, LLC\n3.4"
341,$59K-$125K (Glassdoor est.),23andMe\n4.0


In [6]:
df1 = df['Company Name'].str.split(pat='\n',expand=True)

In [7]:
df = pd.concat([df, df1],axis=1)
df

,Salary Estimate,Company Name,0,1
0,$53K-$91K (Glassdoor est.),Tecolote Research\n3.8,Tecolote Research,3.8
1,$63K-$112K (Glassdoor est.),University of Maryland Medical System\n3.4,University of Maryland Medical System,3.4
2,$80K-$90K (Glassdoor est.),KnowBe4\n4.8,KnowBe4,4.8
3,$56K-$97K (Glassdoor est.),PNNL\n3.8,PNNL,3.8
4,$86K-$143K (Glassdoor est.),Affinity Solutions\n2.9,Affinity Solutions,2.9
...,...,...,...,...
338,$79K-$127K (Glassdoor est.),SoftBank Robotics\n3.8,SoftBank Robotics,3.8
339,Employer Provided Salary:$120K-$140K,SkySync\n5.0,SkySync,5.0
340,$65K-$113K (Glassdoor est.),"DatamanUSA, LLC\n3.4","DatamanUSA, LLC",3.4
341,$59K-$125K (Glassdoor est.),23andMe\n4.0,23andMe,4.0


In [8]:
df.drop(axis=1, columns=['Company Name', 1],inplace=True)

In [9]:
df.rename(columns={0:"Company Name"},inplace=True)
df

,Salary Estimate,Company Name
0,$53K-$91K (Glassdoor est.),Tecolote Research
1,$63K-$112K (Glassdoor est.),University of Maryland Medical System
2,$80K-$90K (Glassdoor est.),KnowBe4
3,$56K-$97K (Glassdoor est.),PNNL
4,$86K-$143K (Glassdoor est.),Affinity Solutions
...,...,...
338,$79K-$127K (Glassdoor est.),SoftBank Robotics
339,Employer Provided Salary:$120K-$140K,SkySync
340,$65K-$113K (Glassdoor est.),"DatamanUSA, LLC"
341,$59K-$125K (Glassdoor est.),23andMe


In [10]:
df2 = df
df1 = df2['Salary Estimate'].str.rstrip('(Glassdoor est.)')
df.drop(axis=1, columns=['Salary Estimate'],inplace=True)
df = pd.concat([df, df1],axis=1)

In [11]:
df.head()

,Company Name,Salary Estimate
0,Tecolote Research,$53K-$91K
1,University of Maryland Medical System,$63K-$112K
2,KnowBe4,$80K-$90K
3,PNNL,$56K-$97K
4,Affinity Solutions,$86K-$143K


In [12]:
mask = df['Company Name'].str.contains('LLC').value_counts()

In [13]:
df.drop(mask, inplace=True)

In [14]:
pd.set_option('display.max_rows',None,'display.max_columns',None)

In [15]:
To_remove_lst = [' .LLC', ' INC.', ' LLC', ' ,INC', ' INC', ' Inc.', ', Inc.', ', Inc']

df['Company Name'] = df['Company Name'].str.replace('|'.join(To_remove_lst), '')

To_remove_lst = [' Inc.', ', Inc.', ', Inc', '. Inc']

df['Company Name'] = df['Company Name'].str.replace('|'.join(To_remove_lst), ',')
df['Company Name'] = df['Company Name'].str.upper()


In [16]:
df

,Company Name,Salary Estimate
0,TECOLOTE RESEARCH,$53K-$91K
1,UNIVERSITY OF MARYLAND MEDICAL SYSTEM,$63K-$112K
2,KNOWBE4,$80K-$90K
3,PNNL,$56K-$97K
4,AFFINITY SOLUTIONS,$86K-$143K
5,CYRUSONE,$71K-$119K
6,CLEARONE ADVANTAGE,$54K-$93K
8,ROCHESTER REGIONAL HEALTH,$38K-$84K
9,<INTENT>,$120K-$160K
10,WISH,$126K-$201K


In [17]:
df.to_csv('cleaned_glassdoor.csv', index=False)